### Load Packages

In [1]:
import tensorflow as tf
import os
os.chdir('/Users/hollowayp/paltas')
from paltas.Analysis import dataset_generation, loss_functions, conv_models


2023-07-28 15:42:04.126973: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numba/core/decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)


### Load Model

In [3]:
model_weights = '/Users/hollowayp/paltas/paltas/example_network_weights/path_to_model_weights_200-5566125.00.h5'
EPOCH = 200
img_size = (160,160,1)
random_seed = 2
batch_size = 5
flip_pairs = None
NORM_IMAGES = True
LOG_NORM_IMAGES = False
learning_params = ['main_deflector_parameters_theta_E',
                    'main_deflector_parameters_gamma1','main_deflector_parameters_gamma2',
                    'main_deflector_parameters_gamma','main_deflector_parameters_e1',
                    'main_deflector_parameters_e2','main_deflector_parameters_center_x',
                    'main_deflector_parameters_center_y',
                    'subhalo_parameters_sigma_sub']
learning_params_print = [r'$\theta_\mathrm{E}$',r'$\gamma_1$',r'$\gamma_2$',r'$\gamma_\mathrm{lens}$',r'$e_1$',
                         r'$e_2$',r'$x_\mathrm{lens}$',r'$y_\mathrm{lens}$']
                         #r'x_{\mathrm{src}}',r'y_{\mathrm{src}}',r'R_{src}']
log_learning_params = []
num_params = len(learning_params+log_learning_params)
loss_type = 'full'
model_type = 'xresnet34'

tf.random.set_seed(random_seed)

if loss_type == 'full':
    num_outputs = num_params + int(num_params*(num_params+1)/2)
    loss_func = loss_functions.FullCovarianceLoss(num_params)

elif loss_type == 'diag':
    num_outputs = 2*num_params
    loss_func = loss_functions.DiagonalCovarianceLoss(num_params)

if model_type == 'xresnet101':
    model = conv_models.build_xresnet101(img_size,num_outputs)
if model_type == 'xresnet34':
    model = conv_models.build_xresnet34(img_size,num_outputs)
model.load_weights(model_weights)
#model.load_weights(model_weights,by_name=True,skip_mismatch=True)


In [4]:
model

### Make Metrics

In [1]:
file_prefix = '../../STRIDES14results/july13_src_info/'
validation_prefix = '../../STRIDES14results/may22_2023/'
already_computed = False


if already_computed:
    y_test = np.load(file_prefix+'val'+str(EPOCH)+'_y_test.npy')
    y_pred = np.load(file_prefix+'val'+str(EPOCH)+'_y_pred.npy')
    std_pred = np.load(file_prefix+'val'+str(EPOCH)+'_std_pred.npy')
    predict_samps = np.load(file_prefix+'val'+str(EPOCH)+'_predict_samps.npy')
    prec_pred = np.load(file_prefix+'val'+str(EPOCH)+'_prec_pred.npy')

else:
    npy_folder_test = validation_prefix + 'validate/'
    tfr_test_path = os.path.join(npy_folder_test,'data.tfrecord')
    input_norm_path = file_prefix+'norms_5e-4_1e3.csv'

    y_test, y_pred, std_pred, prec_pred, predict_samps = mcmc_utils.gen_network_predictions(npy_folder_test,
        input_norm_path,learning_params,loss_func,model,samples=True,shuffle=False,
        norm_images=NORM_IMAGES,log_norm_images=LOG_NORM_IMAGES)
    
    np.save(file_prefix+'val'+str(EPOCH)+'_y_test.npy',y_test)
    np.save(file_prefix+'val'+str(EPOCH)+'_y_pred.npy',y_pred)
    np.save(file_prefix+'val'+str(EPOCH)+'_std_pred.npy',std_pred)
    np.save(file_prefix+'val'+str(EPOCH)+'_predict_samps.npy',predict_samps)
    np.save(file_prefix+'val'+str(EPOCH)+'_prec_pred.npy',prec_pred)
df = pd.read_csv('../../STRIDES14results/may22_2023/validate/metadata.csv')
plt.hist(df['point_source_parameters_num_images'].to_numpy())
plt.title("Num PS Images in Validation Set")
num_images = df['point_source_parameters_num_images'].to_numpy()
np.sum(num_images == 5) / np.size(num_images)
#print(np.where(num_images == 3)[0])

#df['point_source_parameters_x_image_2'].to_numpy()[30]
# find a way to exclude 3 / 5 image configurations (they most likely come from numerical limitations of lenstronomy)
# probably should train on quads only, find a way to evaluate P(xi_k|Omega_int) numerically like Sherry suggested...
    # you have a histogram of probably density from 500,000 images, bin it very finely
    # given a value, assign it to a bin, and then ask for what probably density that bin has 
    # this is an approximate way to evaluate the p(xi_k|Omega_int) without assigning a functional form...
    # this also motivates the move to jax to make things faster (sampling not analytical)
    # is jax / numpyro working on sherlock?
Print table of metrics in latex format

file_prefix = '../../STRIDES14results/may22_2023/'
# indices of interest
iofi = [0,1,2,3,4,5,6,7,8,9,10]
visualization_utils.table_metrics(y_pred[:,iofi],y_test[:,iofi],std_pred[:,iofi],
    file_prefix+'validation_set.txt')
f = open(file_prefix+'validation_set.txt', 'r')
file_contents = f.read()
print(file_contents)

SyntaxError: invalid syntax (1235189569.py, line 42)